                                                    LAB ASSIGNMENTS
                                            Module 2: Machine Learning


Ensemble Method- Random Forest

2. Given is the data Hepatitis C data.csv.
The data description file has been provided.
Binarize the “Category” variable as blood donors vs. Hepatitis C patients (including its progress ('just' Hepatitis C, Fibrosis, Cirrhosis).
Do the following with the dataset:
A. Impute the values in the data with any options of your choice
B. Balance the data with SMOTE / ADASYN method (your choice)
C. Do the train_test_split of the data with test_size=0.1
D. On the train set (90% data) do the following:
a. Randomized / Grid Search CV with Random Forest
b. Randomized / Grid Search CV with XGBoost
c. Randomized / Grid Search CV with SVC-RBF
E. Do .predict_proba(test set) with each of the tuned CV models obtained from (a), (b) and (c) and check for best “roc_auc”

In [1]:
#pip install imbalanced-learn


In [2]:
#pip install pandas scikit-learn imbalanced-learn xgboost


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score



In [4]:
#pip install --upgrade scikit-learn


In [5]:
#pip install --upgrade imbalanced-learn


In [6]:


# Load the dataset
data = pd.read_csv(r'D:\1_AJINKYA_PAWAR\3_SOFTWARE JOB\1_CDAC\3_AI_Course\Lab_Assignment_Module_2\2_5_EnsembleMethod_RandomForest_2Hrs\HepatitisCdata.csv')

data

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,611,3=Cirrhosis,62,f,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
611,612,3=Cirrhosis,64,f,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
612,613,3=Cirrhosis,64,f,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
613,614,3=Cirrhosis,46,f,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0


In [7]:


# Step 3: Binarize the "Category" variable
data['Category'] = data['Category'].apply(lambda x: 1 if x != 'blood donor' else 0)

# Step 4: Impute missing values (you can choose other strategies if needed)
imputer = SimpleImputer(strategy='mean')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Step 5: Balance the data using SMOTE (or ADASYN if you prefer)
X = data.drop('Category', axis=1)
y = data['Category']
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 6: Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)

# Step 7: Tune Random Forest, XGBoost, and SVC-RBF models using Randomized Search CV
# Random Forest
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}
rf_model = RandomForestClassifier(random_state=42)
rf_cv = RandomizedSearchCV(rf_model, rf_params, n_iter=10, scoring='roc_auc', cv=5, random_state=42)
rf_cv.fit(X_train, y_train)

# XGBoost
xgb_params = {
    'learning_rate': [0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 150, 200],
    'subsample': [0.8, 0.9, 1.0],
}
xgb_model = XGBClassifier(random_state=42)
xgb_cv = RandomizedSearchCV(xgb_model, xgb_params, n_iter=10, scoring='roc_auc', cv=5, random_state=42)
xgb_cv.fit(X_train, y_train)

# SVC-RBF
svc_params = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 10],
    'kernel': ['rbf']
}
svc_model = SVC(probability=True, random_state=42)
svc_cv = RandomizedSearchCV(svc_model, svc_params, n_iter=10, scoring='roc_auc', cv=5, random_state=42)
svc_cv.fit(X_train, y_train)

# Step 8: Calculate "roc_auc" for each model on the test set
rf_pred_prob = rf_cv.predict_proba(X_test)[:, 1]
xgb_pred_prob = xgb_cv.predict_proba(X_test)[:, 1]
svc_pred_prob = svc_cv.predict_proba(X_test)[:, 1]

rf_roc_auc = roc_auc_score(y_test, rf_pred_prob)
xgb_roc_auc = roc_auc_score(y_test, xgb_pred_prob)
svc_roc_auc = roc_auc_score(y_test, svc_pred_prob)

print(f"Random Forest ROC AUC: {rf_roc_auc}")
print(f"XGBoost ROC AUC: {xgb_roc_auc}")
print(f"SVC-RBF ROC AUC: {svc_roc_auc}")


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'm'